# 검증세트
테스트 세트는 어디까지나 모델을 완성하고 마지막에 딱 한 번만 사용하는 것이 좋다.  
일반화 성능을 위하여 테스트 세트를 지양하는 것이다.  
  
테스트 세트를 사용하지 않으면 모델이 과대적합인지 과소적합인지 판단할 수 없다.  
테스트 세트를 사용하지 않고 이를 측정하는 방법은 훈련 세트를 또 나누는 것이다.  
이 데이터를 검증세트라고 부른다.  
  
전체 데이터중 20%를 테스트 세트로 만들고 남은 80%중 20%를 또 떼어 내서 검증세트로 만든다.  
훈련세트로 모델을 훈련하고 검증 세트로 모델을 평가한다.  
그리고 마지막에 테스트 세트에서 최종 점수를 평가한다.  

In [1]:
# 데이터 가져오기
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [2]:
# class 열을 타깃으로 사용하고 나머지는 특성 배열에 저장
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
# 훈련세트와 테스트 세트를 분리
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [4]:
# 다시, 검증세트를 만듦, 검증세트는 train_input의 약 20%
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state = 42)

In [5]:
# 훈련세트와 검증세트 크기를 확인
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


원래 5197개였던 훈련세트가 4157개로 줄고 검증세트는 1040개가 되었다.

In [6]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


# 교차검증
쉽게말해 훈련세트 A B C 를 만들고, A와 B로 훈련한다음 C로 검증을 하고  
그 다음 B와 C로 훈련을 하고 A로 검증을 하고 A와 C로 훈련한 다음 B로 훈련하는 것과 같다.  
위와 같이 훈련세트가 3개면 3폴드 교차검증이다.  
보통은 5폴드나 10폴드 교차검증을 많이 사용한다.  
사이킷런에는 cross_validate()라는 교차검증 함수가 있다.  
이것을 사용해 보자

In [7]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.02098274, 0.01666546, 0.01390123, 0.01116204, 0.01057529]), 'score_time': array([0.00224209, 0.00172853, 0.00179195, 0.00175428, 0.00162768]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [8]:
# 교차검증을 통해 얻을 수 있는 최상의 검증점수
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [9]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [10]:
# 10 폴드 교차 검증을 수행하면 다음과 같다.
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


# 하이퍼파라미터 튜닝
머신러닝 모델이 학습하는 파라미터를 모델 파라미터라고 부른다면  
모델이 학습할 수 없어서 사용자가 지정해야만 하는 파라미터를 하이퍼 파라미터라고 한다.  


In [11]:
# 0.0001씩 증가하는 5개의 값을 시도
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [12]:
# 그리드 서치 객체 생성
gs = GridSearchCV(DecisionTreeClassifier(random_state =42), params, n_jobs = -1)

In [13]:
# 훈련
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [14]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [16]:
# 최상의 매개변수 조합
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [18]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [19]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [20]:
params={'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
        'max_depth' : range(5, 20, 1),
        'min_samples_split': range(2, 100, 10)}

In [21]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs = -1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [23]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [24]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


# 렌덤서치
매개변수의 값이 수치일 때 값의 범위나 간격을 미리 정하기 어려울 수 있다.  
이럴 때 랜덤 서치를 사용하면 좋다.  
랜덤서치에는 매개변수 값의 목록이 아닌 매개변수를 샘플링 할 수 있는 확률분포 객체를 전달한다.  
먼저 싸이파이에서 2개의 확률 분포 클래스를 임포트 한다.  

# 싸이파이
적분, 보간, 선형대수, 확률 등을 포함한 수치 게산 전용 라이브러리

In [26]:
# 아래 두 클래스는 모두 주어진 범위에서 고르게 값을 뽑는다.
from scipy.stats import uniform, randint

In [27]:
rgen = randint(0, 10)
rgen.rvs(10)

array([8, 6, 9, 4, 4, 8, 4, 4, 8, 7])

In [28]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([117,  94, 105,  91, 102, 108, 102,  89,  96,  96]))

In [29]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.47670146, 0.49152223, 0.23414866, 0.30550973, 0.86479101,
       0.4803995 , 0.62458683, 0.50020608, 0.67295594, 0.30162231])

In [30]:
# 탐색할 매개변수 범위
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25)
          }

In [31]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state= 42), params, n_iter = 100, n_jobs = -1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fa957cf05e0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7fa956fda070>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fa956fbf730>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7fa956fda790>},
                   random_state=42)

In [32]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [33]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [34]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86


# 정리

테스트 세트는 최종 모델을 선택할 떄까지 사용해선 안 된다.  
테스트 세트 없이 모델을 평가하려면 검증세트가 필요하다.(개발세트라고도 함)  
검증세트가 작으면 데이터를 어떻게 나누었느냐에 따라 점수가 둘쭉날쭉한다.  
훈련 모델의 성능을 안정적으로 평가하기 위해, 검증 세트를 한 번 나누는 것이 아닌  
여러번 나누어 실행함으로써 안정성을 높인다.  
이를 교차검증이라 한다.  
보통 5~10등분을 한다.  